This is where the metadata, feature space, and genres are read in and stored in the "songs" dictionary by `track_id`.

In [1]:
import sklearn
from sklearn.feature_extraction import DictVectorizer
from sklearn.svm import LinearSVC
from random import sample as rand_sample


In [2]:
def read_data(file):
    '''
    description

    :param file:        file we're reading in with the data (.csv)

    :return             a 'songs' dictionary
                        key: track_id
                        value(s): a 'data' dictionary, which contains:
                            metadata,
                            label,
                            features,
                            genres
    '''
    
    songs = {}

    with open(file, 'r') as f:

        for song in f:
            
            line = song.split(',')
                        
            # ignore first line + ensure that label exists
            if line[0] != 'label' and line[0]:
                label = int(line[0])
                metadata, data = {}, {}
                genres = []
                track_id = line[1]
                
                features = {}
                features['artist_popularity'] = line[4]
                features['artist_followers'] = line[5]

                metadata['artist_id'] = line[2]
                metadata['artist_name'] = line[3]
                
                # check if genre field has multiple genres or just one
                if line[6]:
                    genres.append(line[6][1:]) if '"' in line[6] else genres.append(line[6])

                count = 0
                
                # if '"' present or next value is a string only containing alphabets,
                # then add to genres list. set count to i+1 when you reach last genre
                for i in range(7, len(line)):
                    if '"' in line[i]:
                        genres.append(line[i][:-1])
                        count = i+1
                        break
                    if line[i].isalpha():
                        genres.append(line[i])
                
                # single or no genres, get all other features
                if count == 0:
                    features['instrumentalness'] = float(line[7])
                    features['duration_ms'] = float(line[8])
                    features['time_signature'] = float(line[9])
                    features['acousticness'] = float(line[10])
                    features['speechiness'] = float(line[11])
                    features['energy'] = float(line[12])
                    features['loudness'] = float(line[13])
                    features['tempo'] = float(line[14])
                    features['key'] = float(line[15])
                    features['valence'] = float(line[16])
                    features['danceability'] = float(line[17])
                    features['liveness'] = float(line[18][:-1])
                
                # multiple genres, get all other features using count
                else:
                    features['instrumentalness'] = float(line[count])
                    features['duration_ms'] = float(line[count+1])
                    features['time_signature'] = float(line[count+2])
                    features['acousticness'] = float(line[count+3])
                    features['speechiness'] = float(line[count+4])
                    features['energy'] = float(line[count+5])
                    features['loudness'] = float(line[count+6])
                    features['tempo'] = float(line[count+7])
                    features['key'] = float(line[count+8])
                    features['valence'] = float(line[count+9])
                    features['danceability'] = float(line[count+10])
                    features['liveness'] = float(line[count+11][:-1])
                
                # add metadata, features, genres, and label to data
                data['metadata'] = metadata
                data['features'] = features
                data['genres'] = genres
                data['label'] = label
                
                # add data to songs by track_id
                songs[track_id] = data

    return songs

This is where genres are read in, with each genre getting a unique `genre_id`.

In [3]:
def isFloat(string):
    '''
    description

    :param string:      the string we're testing

    :return             True if the string is a float, False otherwise
    '''
    try:
        float(string)
        return True
    except ValueError:
        return False

def read_genres(file):
    '''
    description
    
    :param file:        file we're reading in with the data (.csv)
    
    :return             a 'genres' dictionary
                        key: genre
                        value: unique genre_id
    '''
    genre_mapping = {}
    genre_id = 0
    
    with open(file, 'r') as f:

        for song in f:

            line = song.split(',')

            if line[0]:

                genres = []
                
                # check if genres field is empty or contains one/multiple genres
                if line[6]:
                    genres.append(line[6][1:]) if '"' in line[6] else genres.append(line[6])

                count = 0
                
                # account for multiple genres
                for i in range(7, len(line)):
                    if '"' in line[i]:
                        genres.append(line[i][:-1])
                        count = i+1
                        break
                    if line[i].isalpha():
                        genres.append(line[i])
                
                # for genres in the genres list, ensure that the genre is not a float, create unique genre_id
                for genre in genres:
                    if (genre not in genre_mapping) and ('"' not in genre) and (not isFloat(genre)):
                        genre_mapping[genre] = genre_id
                        genre_id += 1
        
    return genre_mapping

This is where the data is actually read in. We read in the `likes.csv` + `dislikes.csv` data, and merge them into one dictionary, `song_data`.

In [4]:
song_data = read_data('data.csv')
genres = read_genres('data.csv')
feature_names = tuple(next(iter(song_data.values()))['features'].keys())

In [5]:
song_data

{'7B1h0Com19ccdjU8ukJdD4': {'features': {'acousticness': 0.617,
   'artist_followers': '0.064619',
   'artist_popularity': '0.5',
   'danceability': 0.549,
   'duration_ms': 0.092621861,
   'energy': 0.566,
   'instrumentalness': 0.814,
   'key': 0.909090909,
   'liveness': 0.0762,
   'loudness': 0.808119755,
   'speechiness': 0.0427,
   'tempo': 0.49781836,
   'time_signature': 0.8,
   'valence': 0.122},
  'genres': ['british jazz', 'ninja', 'nu jazz'],
  'label': 1,
  'metadata': {'artist_id': '7sYipTRgDXS2JVOPEhRutx',
   'artist_name': 'Portico Quartet'}},
 '3LCT66cdTjGY3FvYcXF8m7': {'features': {'acousticness': 0.487,
   'artist_followers': '0.024933',
   'artist_popularity': '0.46',
   'danceability': 0.592,
   'duration_ms': 0.079964409,
   'energy': 0.607,
   'instrumentalness': 0.807,
   'key': 0.272727273,
   'liveness': 0.138,
   'loudness': 0.767105164,
   'speechiness': 0.0423,
   'tempo': 0.645069949,
   'time_signature': 0.8,
   'valence': 0.176},
  'genres': ['chillwave'

In [6]:
# Sanity check
l, dl = 0, 0
for x in song_data.values():
    if x['label'] == 1: l += 1
    else: dl += 1
print(' likes:', l, '\n', 'dislikes:', dl)

 likes: 1919 
 dislikes: 3790


In [7]:
genres

{'abstract': 246,
 'abstract beats': 132,
 'abstract hip hop': 440,
 'acid house': 127,
 'acid idm': 51,
 'acid jazz': 111,
 'acid techno': 203,
 'acoustic pop': 397,
 'acousticness': 2,
 'adoracao': 407,
 'adult standards': 67,
 'afro house': 32,
 'afrobeat': 115,
 'afropop': 116,
 'alaska indie': 348,
 'alberta country': 347,
 'album rock': 173,
 'alternative ccm': 410,
 'alternative country': 336,
 'alternative dance': 29,
 'alternative hip hop': 92,
 'alternative metal': 236,
 'alternative pop': 276,
 'alternative rock': 103,
 'ambient': 42,
 'ambient fusion': 368,
 'ambient idm': 271,
 'anthem worship': 390,
 'art pop': 75,
 'artist_genres': 0,
 'aussietronica': 58,
 'austindie': 128,
 'australian alternative rock': 170,
 'australian country': 324,
 'australian dance': 186,
 'australian indie': 100,
 'australian indigenous': 345,
 'australian pop': 315,
 'australian rock': 346,
 'baile pop': 490,
 'balearic': 73,
 'ballet class': 163,
 'bass music': 22,
 'bass trap': 250,
 'bass t

In [8]:
feature_names

('time_signature',
 'energy',
 'liveness',
 'tempo',
 'loudness',
 'instrumentalness',
 'acousticness',
 'duration_ms',
 'speechiness',
 'valence',
 'key',
 'artist_followers',
 'danceability',
 'artist_popularity')

In [9]:
class Classifier(object):
    def __init__(self, algorithm, train_data):
        '''
        description
        
        :param algorithm:   the name of the learning algorithm to use
        :param train_data:  training data, dict with following relevant keys: {
                                    features: { feature_name: feature_value},
                                    label: label_value
                            }
        
        :return             Classifier instance
        '''
        
        x_train, y_train = [x['features'] for x in train_data.values()], [x['label'] for x in train_data.values()]
     
        self.algorithm = algorithm
        
        if algorithm == 'svm':
            
            self.vectorizer = DictVectorizer(sort=True)
            
            self.svm = LinearSVC(penalty='l2', loss='hinge')
            self.svm.fit(self.vectorizer.fit_transform(x_train), y_train)
        
        else: raise ValueError('unsupported algorithm: ' + algorithm)
    
    def validate(self, validation_data):
        '''
        Predicts on all instances in validation_data. Returns accuracy.
        '''
        num_instances = len(validation_data)
        x_test, y_test = [x['features'] for x in validation_data.values()], [x['label'] for x in validation_data.values()]
        
        correct = 0
        for i in range(num_instances):
            if self.predict(x_test[i]) == y_test[i]: correct += 1
        
        return correct / num_instances
                
    def predict(self, x):
        if self.algorithm == 'svm': return self.predict_SVM(x)
        raise ValueError('not implemented')
    
    def predict_SVM(self, x):
        x = self.vectorizer.transform([x])
        return self.svm.predict(x)[0]
    

In [10]:
def filter_features(data, discard):
    '''
    Filters out features from data. Does not modify passed-in object (creates a copy).
    
    :param data:        dict of data, same format as song_data
    :param discard:     feature names to discard
                        
    :return             copy of data, with filtered features
    '''
    out = data.copy()
    
    for id_ in data:
        out[id_]['features'] = { k:v for k, v in out[id_]['features'].items() if k not in discard }
    
    return out

def split_data(data, p):
    '''
    Splits data into training and validation sets for simple classification.
    
    :param data:    complete labeled data
    :param p:       proportion of data to use for validation
                    
    :return         train_data, validation_data
    '''
    validation_labels = rand_sample(list(data), int(len(data) * p))
    
    validation_data = {k:data[k] for k in validation_labels}
    train_data = {k:v for k, v in data.items() if k not in validation_data}

    return train_data, validation_data


In [11]:
def run_basic_svm(data, p, discard=None):
    '''
    Trains svm on song_data (with or without filtering features), splits into train/validation,
    and returns accuracy on validation data.
    '''
    if discard: data = filter_features(data, discard)
    
    train_data, validation_data = split_data(data, p)
    
    clf = Classifier('svm', train_data)
    
    return clf.validate(validation_data)

In [12]:
run_basic_svm(song_data, 0.2)

/Users/adi/repos/spotify-music-discovery/venv/lib/python3.5/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.9246275197195443